In [2]:
import pandas as pd
import openpyxl
PATH_TO_EXCEL = "/Users/prashantgaurav/Projects/KaajAi/receipts/Bank-Statement-Template-1-TemplateLab.xlsx"



In [10]:
import openpyxl
from openpyxl_image_loader import SheetImageLoader

pxl_doc = openpyxl.load_workbook(PATH_TO_EXCEL)
sheet = pxl_doc[pxl_doc.sheetnames[0]]

#calling the image_loader
image_loader = SheetImageLoader(sheet)

cell_images = {}

# Iterate over all cells in the sheet
for row in sheet.iter_rows():
    for cell in row:
        if image_loader.image_in(cell.coordinate):
            image = image_loader.get(cell.coordinate)
            cell_images[cell.coordinate] = image

# Print the coordinates of cells with images and display the images
for cell_coord, image in cell_images.items():
    print(f"Image found in cell: {cell_coord}")
    image.show()  # This will display the image using the default image viewer

Image found in cell: A1


In [5]:
type(image_loader)

openpyxl_image_loader.sheet_image_loader.SheetImageLoader

In [17]:
file = pd.read_excel(PATH_TO_EXCEL).fillna('').to_string(index=False)

In [92]:
file

'              Unnamed: 0                                                                    Unnamed: 1 Unnamed: 2 Unnamed: 3              Unnamed: 4           Unnamed: 5 Unnamed: 6 Unnamed: 7         Unnamed: 8\n                         231 Valley Farms Street\\nSanta Monica, CA 90403\\nfirstcitizensbank@domain.com                                                                                                            \n                                                                                                                                                                                                                  \n                                                                                                                                                     STATEMENT OF ACCOUNT                                         \n                                                                                                                                                     

In [96]:
import openai
import os
client = openai.Client(api_key=os.environ.get("OPENAI_API_KEY"))

prompt = """I'm going to give you the contents of a pdf of a bank statement. The text containes transaction records along with other things.
you job is to parse each transaction record and return the details like date, description, amount and type - credit or debit. Return these information in a tabular format. 
Create a seperate column in the returned table named type whose value will be credit or debit."""

docs = [{"role": "system", "content": prompt + '\n' + file}]
response = client.chat.completions.create(
                model="gpt-4",
                messages=docs
            )

In [97]:
resp = response.choices[0].message.content

In [98]:
print(resp)

| Date        | Description         | Amount | Type  |
|-------------|---------------------|--------|-------|
| 05/26/2024  | Payment - Credit Card | 5400  | Debit |
| 05/21/2024  | Payment - Insurance   | 3000  | Debit |
| 05/18/2024  | Account Transfer In | 500000 | Credit |
| 05/16/2024  | Cheque Deposite      | 10000 | Credit |
| 05/09/2024  | Payment - Electricity | 1500  | Debit |
| 05/03/2024  | Payment - Water Utility | 600  | Debit |
| 05/03/2024  | Payment - Car Loan | 3500 | Debit |
| 05/01/2024  | Account Transfer Out | 80000 | Debit |


In [88]:
def parse_response_table(table):
    lines = resp.strip().split("\n")
    table = [line.split('|') for line in lines]
    final_table = []
    for line in table:
        new_line = []
        for ele in line:
            ele = ele.replace('-','')
            if ele :
               new_line.append(ele.strip())
        if not new_line:
            continue
        final_table.append(new_line)
    return final_table
x = parse_response_table(table)

In [89]:
x

[['Date', 'Description', 'Amount', 'Type'],
 ['05/26/2024', 'Payment  Credit Card', '5400', 'Debit'],
 ['05/21/2024', 'Payment  Insurance', '3000', 'Debit'],
 ['05/18/2024', 'Account Transfer In', '500000', 'Credit'],
 ['05/16/2024', 'Cheque Deposite', '10000', 'Credit'],
 ['05/09/2024', 'Payment  Electricity', '1500', 'Debit'],
 ['05/03/2024', 'Payment  Water Utility', '600', 'Debit'],
 ['05/03/2024', 'Payment  Car Loan', '3500', 'Debit'],
 ['05/01/2024', 'Account Transfer Out', '80000', 'Debit']]

In [90]:
df = pd.DataFrame(x[1:],columns=x[0])

In [91]:
df

,Date,Description,Amount,Type
0,05/26/2024,Payment Credit Card,5400,Debit
1,05/21/2024,Payment Insurance,3000,Debit
2,05/18/2024,Account Transfer In,500000,Credit
3,05/16/2024,Cheque Deposite,10000,Credit
4,05/09/2024,Payment Electricity,1500,Debit
5,05/03/2024,Payment Water Utility,600,Debit
6,05/03/2024,Payment Car Loan,3500,Debit
7,05/01/2024,Account Transfer Out,80000,Debit
